In [1]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from sklearn.cluster import AgglomerativeClustering

In [2]:
ticker = "ETH-USD"
rolling_wave_length = 20
num_clusters = 4

In [3]:
df = yf.download(ticker, start = '2023-02-20', period = '1d', interval = '30m')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-02-20 00:00:00,1682.337036,1685.916626,1682.136108,1683.656128,1683.656128,79870464
2023-02-20 00:30:00,1683.150391,1684.469116,1678.555420,1679.450195,1679.450195,14720000
2023-02-20 01:00:00,1678.908569,1678.908569,1659.295044,1660.675659,1660.675659,265348096
2023-02-20 01:30:00,1661.284912,1672.333984,1661.284912,1670.891357,1670.891357,149729792
2023-02-20 02:00:00,1670.981201,1676.015015,1669.214233,1675.393555,1675.393555,100302848
...,...,...,...,...,...,...
2023-02-21 10:30:00,1682.242554,1683.005249,1679.478149,1680.191895,1680.191895,13093376
2023-02-21 11:00:00,1680.565186,1682.718994,1677.756226,1682.718994,1682.718994,57121280
2023-02-21 11:30:00,1682.659912,1682.760132,1672.836426,1672.836426,1672.836426,17954304


In [4]:
def calculate_support_resistance(df, rolling_wave_length, num_clusters):
    date = df.index
    # Reset index for merging
    df.reset_index(inplace=True)
    # Create min and max waves
    max_waves_temp = df.High.rolling(rolling_wave_length).max().rename('waves')
    min_waves_temp = df.Low.rolling(rolling_wave_length).min().rename('waves')
    max_waves = pd.concat([max_waves_temp, pd.Series(np.zeros(len(max_waves_temp)) + 1)], axis=1)
    min_waves = pd.concat([min_waves_temp, pd.Series(np.zeros(len(min_waves_temp)) + -1)], axis=1)
    #  Remove dups
    max_waves.drop_duplicates('waves', inplace=True)
    min_waves.drop_duplicates('waves', inplace=True)
    #  Merge max and min waves
    waves = pd.concat([max_waves, min_waves]).sort_index()
    waves = waves[waves[0] != waves[0].shift()].dropna()
    # Find Support/Resistance with clustering using the rolling stats
    # Create [x,y] array where y is always 1
    x = np.concatenate((waves.waves.values.reshape(-1, 1),
                        (np.zeros(len(waves)) + 1).reshape(-1, 1)), axis=1)
    # Initialize Agglomerative Clustering
    cluster = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
    cluster.fit_predict(x)
    waves['clusters'] = cluster.labels_
    # Get index of the max wave for each cluster
    waves2 = waves.loc[waves.groupby('clusters')['waves'].idxmax()]
    df.index = date
    waves2.waves.drop_duplicates(keep='first', inplace=True)
    return waves2.reset_index().waves

In [5]:
support_resistance_levels = calculate_support_resistance(df, rolling_wave_length, num_clusters)

In [20]:
support_resistance_prices = ""
for level in support_resistance_levels.to_list():
    support_resistance_prices += "$ {:.2f}<br>".format(level)
    
print(support_resistance_prices)

$ 1716.49<br>$ 1661.28<br>$ 1694.78<br>$ 1675.25<br>


In [18]:
# Create subplots and mention plot grid size
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.06, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.3, 0.7])

fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], name = "Market data"), row = 1, col = 1)

fig.update_xaxes(
    rangeslider_visible = False,
    rangeselector=dict(
        buttons=list([
            dict(count=5, label="5h", step="hour", stepmode="backward"),
            dict(count=10, label="10h", step="hour", stepmode="backward"),
            dict(count=15, label="15h", step="hour", stepmode="backward"),
            dict(count=20, label="20h", step="hour", stepmode="backward"),
            dict(count=25, label="25h", step="hour", stepmode="backward"),
            dict(step="all")])))

support_resistance_prices = ""
for level in support_resistance_levels.to_list():
    support_resistance_prices += "$ {:.2f}".format(level)

fig.add_annotation(text=support_resistance_prices,
                align='right',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=1.15,
                y=0.9)
    
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], showlegend=False), row=2, col=1)
fig.update_layout(
    title=go.layout.Title(
        text=ticker,
        xref="paper",
        x=0))


fig.show()